# Websockets

#### Further References:

- https://docs.polymarket.com/#websocket-api

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import asyncio
import json
import logging
from asyncio import Queue
from collections import deque, defaultdict
import anyio
import orjson
import websockets
import hashlib
import json
from pathlib import Path
import copy
import orjson
import pandas as pd
import plotly.express as px
import polars as pl

Want to Get Best Bid and Ask prices including Volume for a given markt and a specific order book (outcome).

## 0. Setup

In [28]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [29]:
# dict of tasks that are running in the background
background_tasks = set()

# Queue for message processing
message_queue = Queue()

## 1. Subscribe to market Channel

In [30]:
# Initialize a deque with a maximum length of 1 to store the latest order book
order_book_store = deque(maxlen=1)

In [31]:
# WebSocket URL
WSS_URL = "wss://ws-subscriptions-clob.polymarket.com/ws/market"

# send Ping manually every 10 seconds
PING_INTERVAL = 10

In [32]:
subscription_message = {
    "assets_ids": [
        "53324254455450563344400636027576535376151041372106632125382807757609541279524",
    ],
    "type": "Market",
}

In [33]:
async def ping(ws: object) -> None:
    """Ping the server every PING_INTERVAL seconds to keep the connection alive."""
    try:
        while True:
            await asyncio.sleep(PING_INTERVAL)
            await ws.ping()
            logger.info("Ping sent")
    except websockets.exceptions.ConnectionClosed as e:
        logger.exception(f"Connection closed during ping loop: {e.code} - {e.reason}")

In [34]:
def update_order_book(order_book: dict, price_change: dict) -> dict:
    # make a copy of the order book
    new_order_book = copy.deepcopy(order_book)
    changes = price_change["changes"]
    for change in changes:
        change_price = change["price"]
        change_size = change["size"]
        side = change["side"]

        if side == "BUY":
            # Find the corresponding price in the bids (buy orders)
            found = False
            for bid in new_order_book["bids"]:
                if bid["price"] == change_price:
                    bid["size"] = change_size
                    found = True
                    break
            if not found:
                # If the price was not found, insert new bid
                new_order_book["bids"].append(
                    {"price": change_price, "size": change_size},
                )

        elif side == "SELL":
            # Find the corresponding price in the asks (sell orders)
            found = False
            for ask in new_order_book["asks"]:
                if ask["price"] == change_price:
                    ask["size"] = change_size
                    found = True
                    break
            if not found:
                # If the price was not found, insert new ask
                new_order_book["asks"].append(
                    {"price": change_price, "size": change_size},
                )

    # Update the timestamp and hash
    new_order_book["timestamp"] = price_change["timestamp"]

    # Recalculate the hash of the updated order book (simple hash of its JSON representation)
    order_book_str = json.dumps(new_order_book, sort_keys=True)
    new_order_book["hash"] = hashlib.sha256(order_book_str.encode("utf-8")).hexdigest()

    return new_order_book


In [35]:
async def handle_message(message: str) -> None:
    """Handle incoming message and save to JSON."""
    try:
        data = json.loads(message)

        # Check if the message is a list
        if isinstance(data, list):
            # Process each item in the list
            for item in data:
                if isinstance(item, dict):
                    event_type = item.get("event_type")

                    if event_type == "book":
                        new_order_book = item
                        # logger.info(f"Processing book event: {item}")
                        # logger.debug(f"Book data: {json.dumps(item, indent=4)}")

                    elif event_type == "price_change":
                        new_order_book = update_order_book(order_book_store[-1], item)
                        # logger.info(f"Processing price_change event: {item}")
                        # logger.debug(f"Price change data: {json.dumps(item, indent=4)}")
                    else:
                        logger.debug(
                            f"Skipping event with unsupported type: {event_type}",
                        )

                    # add new order book to the store
                    logger.info(f"New order book: {new_order_book}")
                    order_book_store.append(new_order_book)
                else:
                    logger.debug(f"Skipping non-dictionary item: {item}")
        else:
            logger.warning(f"Unexpected data format: {type(data)}")

    except json.JSONDecodeError:
        logger.exception(f"Failed to decode message: {message}.")
    except Exception:
        logger.exception("Unexpected error while handling message.")


In [36]:
async def message_processor() -> None:
    """Process messages from the queue."""
    while True:
        # Get message from the queue
        message = await message_queue.get()
        # Process the message
        await handle_message(message)

In [37]:
async def on_open(ws: object) -> None:
    """Callback function when the connection is opened."""
    logger.info("Connection opened")
    # Send an initial ping message
    await ws.ping()
    logger.info("Initial ping sent")

    # Send subscription message
    try:
        await ws.send(json.dumps(subscription_message))
        logger.info("Subscription message sent")
    except Exception:
        logger.exception("Failed to send subscription message.")
        return

    # Start the ping loop in the background
    ping_task = asyncio.create_task(ping(ws))
    background_tasks.add(ping_task)

    # Start the message processor in the background
    processor_task = asyncio.create_task(message_processor())
    background_tasks.add(processor_task)

    # Main loop to listen for messages
    try:
        while True:
            message = await ws.recv()
            await message_queue.put(
                message,
            )  # Put message into the queue for processing
    except websockets.exceptions.ConnectionClosed as e:
        logger.exception(f"Connection closed: {e.code} - {e.reason}")
    except Exception:
        logger.exception("Error while receiving message.")

In [38]:
async def on_close(ws: object) -> None:
    """Callback function when the connection is closed."""
    logger.info("Closing WebSocket connection...")
    for task in background_tasks:
        task.cancel()
    await ws.close()

In [39]:
# Main WebSocket connection function
async def connect(url: str) -> None:
    """Connect to the WebSocket server."""
    async with websockets.connect(url) as ws:
        await on_open(ws)

In [40]:
# Run the WebSocket connection in Jupyter
task = asyncio.create_task(connect(WSS_URL))
background_tasks.add(task)

# Use await directly in Jupyter, no need to manually start the loop
await task  # This will run the WebSocket connection and handle messages

INFO:root:Connection opened
INFO:root:Initial ping sent
INFO:root:Subscription message sent
INFO:root:New order book: {'market': '0x4e2dd28f54a645ac62743a49116dfae0b6fd22ef4187ee2a340d1346ca612bcf', 'asset_id': '53324254455450563344400636027576535376151041372106632125382807757609541279524', 'timestamp': '1734898288926', 'hash': '4d726915e0bb369942c3aab386757093f312ad6c', 'bids': [{'price': '0.001', 'size': '265050'}, {'price': '0.002', 'size': '200000'}, {'price': '0.003', 'size': '10800'}, {'price': '0.004', 'size': '56010'}, {'price': '0.005', 'size': '63111'}, {'price': '0.007', 'size': '54719.55'}, {'price': '0.008', 'size': '2340'}, {'price': '0.009', 'size': '50883.61'}, {'price': '0.01', 'size': '69962.27'}], 'asks': [{'price': '0.997', 'size': '2200000'}, {'price': '0.99', 'size': '10800'}, {'price': '0.987', 'size': '200000'}, {'price': '0.98', 'size': '5400'}, {'price': '0.963', 'size': '200000'}, {'price': '0.96', 'size': '7500'}, {'price': '0.959', 'size': '7317'}, {'price'

CancelledError: 

Two types of messages are received always in a list. One has the `event_type` as `book` and the other has `price_change`:

**market**:
[{"market":"0x9c66114d2dfe2139325cc7a408a5cd5d2e73b55d919e2141b3a0ed83fc15895d","asset_id":"81337566243073271746608276915310766292424419662449021769243514448183489065187","timestamp":"1733156591667","hash":"5042bd0ec48830d08436e41a6b21d47ba6049601","bids":[{"price":"0.001","size":"11167.23"},{"price":"0.002","size":"1618.59"},{"price":"0.003","size":"16806.72"},{"price":"0.005","size":"52869.22"},{"price":"0.01","size":"9714.44"},{"price":"0.011","size":"2104"},{"price":"0.015","size":"650"},{"price":"0.018","size":"336"},{"price":"0.02","size":"17519.76"},{"price":"0.021","size":"11000"},{"price":"0.028","size":"336"},{"price":"0.03","size":"55817.92"},{"price":"0.031","size":"649.42"},{"price":"0.033","size":"9000"},{"price":"0.04","size":"1235.59"},{"price":"0.041","size":"10000"},{"price":"0.042","size":"2276.11"},{"price":"0.043","size":"5"},{"price":"0.045","size":"10000"},{"price":"0.046","size":"6060.37"},{"price":"0.048","size":"20361.98"},{"price":"0.049","size":"68.96"},{"price":"0.05","size":"59290.82"},{"price":"0.051","size":"1087.78"},{"price":"0.053","size":"5100"},{"price":"0.055","size":"6010"},{"price":"0.056","size":"591.75"},{"price":"0.057","size":"100"},{"price":"0.06","size":"40300"},{"price":"0.061","size":"5200.53"},{"price":"0.065","size":"15000"},{"price":"0.068","size":"168"},{"price":"0.069","size":"1983"},{"price":"0.07","size":"6886.34"},{"price":"0.071","size":"30025.96"},{"price":"0.072","size":"149.49"},{"price":"0.074","size":"9999"},{"price":"0.075","size":"662.12"},{"price":"0.079","size":"1983"},{"price":"0.08","size":"2771.57"},{"price":"0.085","size":"2000"},{"price":"0.089","size":"1983"},{"price":"0.09","size":"150"},{"price":"0.094","size":"7.48"},{"price":"0.097","size":"10080"},{"price":"0.099","size":"808"},{"price":"0.1","size":"18652.45"},{"price":"0.101","size":"12022.56"},{"price":"0.102","size":"10684.14"},{"price":"0.104","size":"1152.77"},{"price":"0.109","size":"808"},{"price":"0.11","size":"120"},{"price":"0.111","size":"1754.75"},{"price":"0.112","size":"130"},{"price":"0.113","size":"280"},{"price":"0.114","size":"293.95"},{"price":"0.115","size":"499.99"},{"price":"0.117","size":"2678.14"},{"price":"0.118","size":"1100"},{"price":"0.12","size":"2583.72"},{"price":"0.121","size":"1033.63"},{"price":"0.123","size":"71"},{"price":"0.124","size":"10080"},{"price":"0.125","size":"164.99"},{"price":"0.127","size":"555"},{"price":"0.13","size":"150"},{"price":"0.133","size":"100"},{"price":"0.137","size":"287.25"},{"price":"0.14","size":"2436.49"},{"price":"0.142","size":"1500"},{"price":"0.143","size":"100"},{"price":"0.15","size":"5256.01"},{"price":"0.151","size":"10600"},{"price":"0.153","size":"250"},{"price":"0.155","size":"499.99"},{"price":"0.156","size":"5010.69"},{"price":"0.157","size":"44499.08"},{"price":"0.16","size":"2000"},{"price":"0.161","size":"11200"},{"price":"0.162","size":"100"},{"price":"0.163","size":"200"},{"price":"0.17","size":"1014.69"},{"price":"0.176","size":"750"},{"price":"0.178","size":"40"},{"price":"0.179","size":"2049.99"},{"price":"0.18","size":"410.02"},{"price":"0.181","size":"1305"},{"price":"0.182","size":"53.33"},{"price":"0.184","size":"112.57"},{"price":"0.187","size":"5000"},{"price":"0.19","size":"50005"},{"price":"0.196","size":"750"},{"price":"0.199","size":"50000"},{"price":"0.2","size":"78526.85"},{"price":"0.201","size":"10745.17"},{"price":"0.202","size":"2042.12"},{"price":"0.203","size":"3000"},{"price":"0.21","size":"50"},{"price":"0.212","size":"264.93"},{"price":"0.214","size":"1234.26"},{"price":"0.216","size":"500"},{"price":"0.217","size":"5818.63"},{"price":"0.218","size":"2616.65"},{"price":"0.22","size":"34179.53"},{"price":"0.222","size":"1000"},{"price":"0.223","size":"4867.89"},{"price":"0.224","size":"746.14"},{"price":"0.225","size":"1000"},{"price":"0.228","size":"4945.6"},{"price":"0.23","size":"58842.59"},{"price":"0.231","size":"877.83"},{"price":"0.232","size":"250"},{"price":"0.235","size":"128"},{"price":"0.236","size":"64.43"},{"price":"0.237","size":"13.43"},{"price":"0.238","size":"20.14"},{"price":"0.24","size":"40"},{"price":"0.241","size":"2000"},{"price":"0.243","size":"1000"},{"price":"0.246","size":"25800"},{"price":"0.247","size":"1337.06"},{"price":"0.248","size":"42.86"}],"asks":[{"price":"0.999","size":"11000026.47"},{"price":"0.998","size":"1000"},{"price":"0.99","size":"442.62"},{"price":"0.989","size":"328.49"},{"price":"0.98","size":"106666"},{"price":"0.979","size":"250000"},{"price":"0.97","size":"300"},{"price":"0.968","size":"20000"},{"price":"0.967","size":"90000"},{"price":"0.96","size":"15.68"},{"price":"0.95","size":"26.46"},{"price":"0.93","size":"16.62"},{"price":"0.92","size":"10"},{"price":"0.915","size":"161.29"},{"price":"0.91","size":"11.66"},{"price":"0.9","size":"101088.36"},{"price":"0.898","size":"8000"},{"price":"0.89","size":"128176.5"},{"price":"0.889","size":"30000"},{"price":"0.87","size":"496.91"},{"price":"0.85","size":"464.58"},{"price":"0.82","size":"50"},{"price":"0.8","size":"85.83"},{"price":"0.784","size":"2000"},{"price":"0.783","size":"2000"},{"price":"0.782","size":"4000"},{"price":"0.781","size":"1000"},{"price":"0.779","size":"5000"},{"price":"0.778","size":"1800"},{"price":"0.75","size":"359.98"},{"price":"0.74","size":"40"},{"price":"0.7","size":"10"},{"price":"0.67","size":"350"},{"price":"0.65","size":"7"},{"price":"0.649","size":"2000"},{"price":"0.64","size":"328.15"},{"price":"0.639","size":"4000"},{"price":"0.638","size":"3600"},{"price":"0.63","size":"9055"},{"price":"0.603","size":"31.73"},{"price":"0.6","size":"719.75"},{"price":"0.585","size":"171"},{"price":"0.578","size":"259.87"},{"price":"0.577","size":"2000"},{"price":"0.56","size":"2011"},{"price":"0.55","size":"5"},{"price":"0.54","size":"330"},{"price":"0.53","size":"6130.43"},{"price":"0.52","size":"39"},{"price":"0.5","size":"6707.42"},{"price":"0.498","size":"1234.8"},{"price":"0.497","size":"397"},{"price":"0.495","size":"22"},{"price":"0.491","size":"50"},{"price":"0.49","size":"11.53"},{"price":"0.489","size":"4000"},{"price":"0.485","size":"348.43"},{"price":"0.483","size":"101.76"},{"price":"0.48","size":"265.56"},{"price":"0.46","size":"15"},{"price":"0.454","size":"30000"},{"price":"0.45","size":"128"},{"price":"0.445","size":"5000"},{"price":"0.443","size":"300"},{"price":"0.441","size":"180"},{"price":"0.439","size":"200"},{"price":"0.433","size":"9000"},{"price":"0.43","size":"1000"},{"price":"0.42","size":"929.05"},{"price":"0.415","size":"200"},{"price":"0.41","size":"1669"},{"price":"0.405","size":"200"},{"price":"0.403","size":"1000"},{"price":"0.4","size":"10069.34"},{"price":"0.399","size":"23912.07"},{"price":"0.398","size":"510.68"},{"price":"0.395","size":"80"},{"price":"0.393","size":"300"},{"price":"0.391","size":"40"},{"price":"0.39","size":"20"},{"price":"0.389","size":"3281.72"},{"price":"0.381","size":"125"},{"price":"0.38","size":"34589.23"},{"price":"0.379","size":"497"},{"price":"0.375","size":"971.58"},{"price":"0.371","size":"1000"},{"price":"0.37","size":"8932.03"},{"price":"0.35","size":"151.68"},{"price":"0.349","size":"15000.06"},{"price":"0.343","size":"10000"},{"price":"0.34","size":"20"},{"price":"0.327","size":"100"},{"price":"0.323","size":"1227.06"},{"price":"0.32","size":"145.78"},{"price":"0.319","size":"2917.4"},{"price":"0.318","size":"19.48"},{"price":"0.31","size":"108.39"},{"price":"0.308","size":"361.66"},{"price":"0.3","size":"560.97"},{"price":"0.29","size":"2000"},{"price":"0.283","size":"57.22"},{"price":"0.279","size":"1000"},{"price":"0.277","size":"10"},{"price":"0.275","size":"200"},{"price":"0.273","size":"50"},{"price":"0.272","size":"700"},{"price":"0.269","size":"435.31"},{"price":"0.268","size":"12"},{"price":"0.263","size":"50"},{"price":"0.26","size":"31650.55"},{"price":"0.259","size":"443.42"}],"event_type":"book"},{"market":"0x9c66114d2dfe2139325cc7a408a5cd5d2e73b55d919e2141b3a0ed83fc15895d","asset_id":"64903093311385616430821497488306433314807585397286521531639186532059591846310","timestamp":"1733156591667","hash":"f08e0e0b3d8f4de91c2510ee42c8538141ebcdca","bids":[{"price":"0.001","size":"11000026.47"},{"price":"0.002","size":"1000"},{"price":"0.01","size":"442.62"},{"price":"0.011","size":"328.49"},{"price":"0.02","size":"106666"},{"price":"0.021","size":"250000"},{"price":"0.03","size":"300"},{"price":"0.032","size":"20000"},{"price":"0.033","size":"90000"},{"price":"0.04","size":"15.68"},{"price":"0.05","size":"26.46"},{"price":"0.07","size":"16.62"},{"price":"0.08","size":"10"},{"price":"0.085","size":"161.29"},{"price":"0.09","size":"11.66"},{"price":"0.1","size":"101088.36"},{"price":"0.102","size":"8000"},{"price":"0.11","size":"128176.5"},{"price":"0.111","size":"30000"},{"price":"0.13","size":"496.91"},{"price":"0.15","size":"464.58"},{"price":"0.18","size":"50"},{"price":"0.2","size":"85.83"},{"price":"0.216","size":"2000"},{"price":"0.217","size":"2000"},{"price":"0.218","size":"4000"},{"price":"0.219","size":"1000"},{"price":"0.221","size":"5000"},{"price":"0.222","size":"1800"},{"price":"0.25","size":"359.98"},{"price":"0.26","size":"40"},{"price":"0.3","size":"10"},{"price":"0.33","size":"350"},{"price":"0.35","size":"7"},{"price":"0.351","size":"2000"},{"price":"0.36","size":"328.15"},{"price":"0.361","size":"4000"},{"price":"0.362","size":"3600"},{"price":"0.37","size":"9055"},{"price":"0.397","size":"31.73"},{"price":"0.4","size":"719.75"},{"price":"0.415","size":"171"},{"price":"0.422","size":"259.87"},{"price":"0.423","size":"2000"},{"price":"0.44","size":"2011"},{"price":"0.45","size":"5"},{"price":"0.46","size":"330"},{"price":"0.47","size":"6130.43"},{"price":"0.48","size":"39"},{"price":"0.5","size":"6707.42"},{"price":"0.502","size":"1234.8"},{"price":"0.503","size":"397"},{"price":"0.505","size":"22"},{"price":"0.509","size":"50"},{"price":"0.51","size":"11.53"},{"price":"0.511","size":"4000"},{"price":"0.515","size":"348.43"},{"price":"0.517","size":"101.76"},{"price":"0.52","size":"265.56"},{"price":"0.54","size":"15"},{"price":"0.546","size":"30000"},{"price":"0.55","size":"128"},{"price":"0.555","size":"5000"},{"price":"0.557","size":"300"},{"price":"0.559","size":"180"},{"price":"0.561","size":"200"},{"price":"0.567","size":"9000"},{"price":"0.57","size":"1000"},{"price":"0.58","size":"929.05"},{"price":"0.585","size":"200"},{"price":"0.59","size":"1669"},{"price":"0.595","size":"200"},{"price":"0.597","size":"1000"},{"price":"0.6","size":"10069.34"},{"price":"0.601","size":"23912.07"},{"price":"0.602","size":"510.68"},{"price":"0.605","size":"80"},{"price":"0.607","size":"300"},{"price":"0.609","size":"40"},{"price":"0.61","size":"20"},{"price":"0.611","size":"3281.72"},{"price":"0.619","size":"125"},{"price":"0.62","size":"34589.23"},{"price":"0.621","size":"497"},{"price":"0.625","size":"971.58"},{"price":"0.629","size":"1000"},{"price":"0.63","size":"8932.03"},{"price":"0.65","size":"151.68"},{"price":"0.651","size":"15000.06"},{"price":"0.657","size":"10000"},{"price":"0.66","size":"20"},{"price":"0.673","size":"100"},{"price":"0.677","size":"1227.06"},{"price":"0.68","size":"145.78"},{"price":"0.681","size":"2917.4"},{"price":"0.682","size":"19.48"},{"price":"0.69","size":"108.39"},{"price":"0.692","size":"361.66"},{"price":"0.7","size":"560.97"},{"price":"0.71","size":"2000"},{"price":"0.717","size":"57.22"},{"price":"0.721","size":"1000"},{"price":"0.723","size":"10"},{"price":"0.725","size":"200"},{"price":"0.727","size":"50"},{"price":"0.728","size":"700"},{"price":"0.731","size":"435.31"},{"price":"0.732","size":"12"},{"price":"0.737","size":"50"},{"price":"0.74","size":"31650.55"},{"price":"0.741","size":"443.42"}],"asks":[{"price":"0.999","size":"11167.23"},{"price":"0.998","size":"1618.59"},{"price":"0.997","size":"16806.72"},{"price":"0.995","size":"52869.22"},{"price":"0.99","size":"9714.44"},{"price":"0.989","size":"2104"},{"price":"0.985","size":"650"},{"price":"0.982","size":"336"},{"price":"0.98","size":"17519.76"},{"price":"0.979","size":"11000"},{"price":"0.972","size":"336"},{"price":"0.97","size":"55817.92"},{"price":"0.969","size":"649.42"},{"price":"0.967","size":"9000"},{"price":"0.96","size":"1235.59"},{"price":"0.959","size":"10000"},{"price":"0.958","size":"2276.11"},{"price":"0.957","size":"5"},{"price":"0.955","size":"10000"},{"price":"0.954","size":"6060.37"},{"price":"0.952","size":"20361.98"},{"price":"0.951","size":"68.96"},{"price":"0.95","size":"59290.82"},{"price":"0.949","size":"1087.78"},{"price":"0.947","size":"5100"},{"price":"0.945","size":"6010"},{"price":"0.944","size":"591.75"},{"price":"0.943","size":"100"},{"price":"0.94","size":"40300"},{"price":"0.939","size":"5200.53"},{"price":"0.935","size":"15000"},{"price":"0.932","size":"168"},{"price":"0.931","size":"1983"},{"price":"0.93","size":"6886.34"},{"price":"0.929","size":"30025.96"},{"price":"0.928","size":"149.49"},{"price":"0.926","size":"9999"},{"price":"0.925","size":"662.12"},{"price":"0.921","size":"1983"},{"price":"0.92","size":"2771.57"},{"price":"0.915","size":"2000"},{"price":"0.911","size":"1983"},{"price":"0.91","size":"150"},{"price":"0.906","size":"7.48"},{"price":"0.903","size":"10080"},{"price":"0.901","size":"808"},{"price":"0.9","size":"18652.45"},{"price":"0.899","size":"12022.56"},{"price":"0.898","size":"10684.14"},{"price":"0.896","size":"1152.77"},{"price":"0.891","size":"808"},{"price":"0.89","size":"120"},{"price":"0.889","size":"1754.75"},{"price":"0.888","size":"130"},{"price":"0.887","size":"280"},{"price":"0.886","size":"293.95"},{"price":"0.885","size":"499.99"},{"price":"0.883","size":"2678.14"},{"price":"0.882","size":"1100"},{"price":"0.88","size":"2583.72"},{"price":"0.879","size":"1033.63"},{"price":"0.877","size":"71"},{"price":"0.876","size":"10080"},{"price":"0.875","size":"164.99"},{"price":"0.873","size":"555"},{"price":"0.87","size":"150"},{"price":"0.867","size":"100"},{"price":"0.863","size":"287.25"},{"price":"0.86","size":"2436.49"},{"price":"0.858","size":"1500"},{"price":"0.857","size":"100"},{"price":"0.85","size":"5256.01"},{"price":"0.849","size":"10600"},{"price":"0.847","size":"250"},{"price":"0.845","size":"499.99"},{"price":"0.844","size":"5010.69"},{"price":"0.843","size":"44499.08"},{"price":"0.84","size":"2000"},{"price":"0.839","size":"11200"},{"price":"0.838","size":"100"},{"price":"0.837","size":"200"},{"price":"0.83","size":"1014.69"},{"price":"0.824","size":"750"},{"price":"0.822","size":"40"},{"price":"0.821","size":"2049.99"},{"price":"0.82","size":"410.02"},{"price":"0.819","size":"1305"},{"price":"0.818","size":"53.33"},{"price":"0.816","size":"112.57"},{"price":"0.813","size":"5000"},{"price":"0.81","size":"50005"},{"price":"0.804","size":"750"},{"price":"0.801","size":"50000"},{"price":"0.8","size":"78526.85"},{"price":"0.799","size":"10745.17"},{"price":"0.798","size":"2042.12"},{"price":"0.797","size":"3000"},{"price":"0.79","size":"50"},{"price":"0.788","size":"264.93"},{"price":"0.786","size":"1234.26"},{"price":"0.784","size":"500"},{"price":"0.783","size":"5818.63"},{"price":"0.782","size":"2616.65"},{"price":"0.78","size":"34179.53"},{"price":"0.778","size":"1000"},{"price":"0.777","size":"4867.89"},{"price":"0.776","size":"746.14"},{"price":"0.775","size":"1000"},{"price":"0.772","size":"4945.6"},{"price":"0.77","size":"58842.59"},{"price":"0.769","size":"877.83"},{"price":"0.768","size":"250"},{"price":"0.765","size":"128"},{"price":"0.764","size":"64.43"},{"price":"0.763","size":"13.43"},{"price":"0.762","size":"20.14"},{"price":"0.76","size":"40"},{"price":"0.759","size":"2000"},{"price":"0.757","size":"1000"},{"price":"0.754","size":"25800"},{"price":"0.753","size":"1337.06"},{"price":"0.752","size":"42.86"}],"event_type":"book"}]

**asset_id**:
[{"asset_id":"81337566243073271746608276915310766292424419662449021769243514448183489065187","changes":[{"price":"0.25","side":"BUY","size":"3581.76"}],"event_type":"price_change","hash":"7c8225597a104453215a960cab672102f3aad3fb","market":"0x9c66114d2dfe2139325cc7a408a5cd5d2e73b55d919e2141b3a0ed83fc15895d","timestamp":"1733156594352"},{"asset_id":"64903093311385616430821497488306433314807585397286521531639186532059591846310","changes":[{"price":"0.75","side":"SELL","size":"3581.76"}],"event_type":"price_change","hash":"67a6de650d55ddf0bed4ced33bb87a9289937991","market":"0x9c66114d2dfe2139325cc7a408a5cd5d2e73b55d919e2141b3a0ed83fc15895d","timestamp":"1733156594352"}]